In [ ]:
import os
import sys
import numpy
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

import matplotlib.pyplot as plt
from IPython import display
from astropy.table import Table,vstack
from desiutil.log import get_logger, DEBUG
from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *
import requests
from timedomain.sp_utils import SkyPortal as sp
import datetime
from astropy.time import Time
import matplotlib.pyplot as plt
from numpy import pi, linspace
from numpy.random import normal
import math
import scipy

In [ ]:
def Lorentz(x,gamma, x0, A):
    L = A*(1/pi)*0.5*gamma/((x-x0)**2 + (0.5*gamma)**2)
    return L

In [ ]:
def continuum(m,x,b):
    cont = m*x + b
    return cont

In [ ]:
r = linspace(3000,10000,8000)

In [ ]:
spec = Lorentz(r, 10, 6562, 5000) + Lorentz(r,10,5007,4000) + Lorentz(r,10, 4852,3500) + Lorentz(r,50, 3835,2500)
# spec = [normal(i, 0.5) for i in spec]

In [ ]:
plt.plot(r,spec)
plt.show()

In [ ]:
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

import matplotlib.pyplot as plt
from IPython import display
from astropy.table import Table,vstack
from desiutil.log import get_logger, DEBUG
from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *
import requests
from timedomain.sp_utils import SkyPortal as sp
import datetime
from astropy.time import Time
date=20210606
tile_petal,group_tid,group_tp, group_night =getMatchedTileid(date)
#Set non-default plot size 
plt.rcParams["figure.figsize"] = (20,6)

#SkyPortal token:
secret_file = "/global/cfs/cdirs/desi/science/td/secrets/desidiff_sp.txt"
with open(secret_file, 'r') as file:
    token = file.read().replace('\n', '')
headers = {'Authorization': f'token {token}'}

filter_name = 'DESIDIFF'
lminb=3700.
lminr=5800.
lmaxr=7580.
lmaxz=9100.

# tile_petal contain subsets of tile/petals whose RA/DEC targets are not contained in other tile/petals from that night
for tps in tile_petal:
    # Check to see if this tile/petal has nothing to subtract.  Skip's IO if this is the case
    if hasNothingToProcess(tps,group_tid,group_tp,group_night):
        continue
    
    # cache spectra to minimize IO
    spectra_list=dict()
    zbest_list = []
    target_list=[]
    ra_dec_list = []
    for tp in tps:
        filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{tp[0]}/{date}/spectra-{tp[1]}-{tp[0]}-thru{date}.fits"
        spectra_list[(tp[0],tp[1])]=read_spectra(filename)
        #To get RA/DEC info for the object
        ra_dec_list.append(Table.read(filename, format='fits',hdu=1, memmap=True)['TARGETID','TARGET_RA', 'TARGET_DEC'])
        zbest = filename.replace('spectra','zbest')
        z = Table.read(zbest, format='fits',hdu=1, memmap=True)['TARGETID','Z','ZERR','ZWARN','SPECTYPE']
        zbest_list.append(z)
    z = vstack(zbest_list)
    ra_dec = vstack(ra_dec_list)
    

    count=0
    # loop over all unique RA/DEC pairs from that night
    for tid, tp, night in zip(group_tid,group_tp,group_night):
        # if this RA/DEC is not in thie tile_petal combination than skip
        if tp[0] not in tps:
            continue
        
        # Eliminate those with no reference night here
        if len(night) == 1:
            continue

        # Obtain METAINFORMATION for this RA/DEC
        #Stores a tuple = (TARGETID, Z, ZERR,ZWARN, SPECTYPE)
        z_data = []
        ra_dec_data = []
        for t in tid:
            tid_ind = list(z['TARGETID']).index(t)
            ra_dec_ind = list(ra_dec['TARGETID']).index(t)
            z_data.append((t, z[tid_ind]['Z'], z[tid_ind]['ZERR'], z[tid_ind]['ZWARN'], z[tid_ind]['SPECTYPE']))
            ra_dec_data.append((t, ra_dec[ra_dec_ind]['TARGET_RA'], ra_dec[ra_dec_ind]['TARGET_DEC']))
        redshift = z_data[0][1] #it seems each set only has one value
        spectype = z_data[0][-1]
        
        # Proceed with a subtraction for this object
        
        # The coadds of the new and reference are constructed from all spectra with all targetid's in tid and all
        # tile/petal combinations in tp, which are cached above
        
        newSpectra=[]
        refSpectra=[]
        for tile,plate in tp:
            spec = spectra_list[(tile,plate)]

            idx = numpy.in1d(night, date)
            ref_night = night[~idx]
            
            newSpectra.append(spec.select(nights=date, targets = tid))
            
            """
            There is a variable night that has all the nights associated with this RA/DEC
            derive the ref_night from that
            """
            
            refSpectra.append(spec.select(nights=ref_night, targets = tid))       
         
        
        newflux, newivar, newwave, newmask = coadd(newSpectra)
        refflux, refivar, refwave, refmask = coadd(refSpectra)

        # renormalize spectra to match each other
        # There is a significant background of spectra that have the same shape but different fluxes
        # This seems to be related to mistaken coordinates of bright sources
        norm = normalization(newflux,newmask, refflux,refmask)

        for key in newflux.keys():
            newflux[key]=newflux[key]/norm
            newivar[key]=newivar[key]*norm**2        
        
        difflux, difivar, difmask, difwave = dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"]), dict.fromkeys(["b", "r", "z"])
        
        difflux = {key: newflux[key] - refflux[key]
                       for key in newflux.keys()}
        difivar = {key: 1./(1./newivar[key] + 1./refivar[key])
                       for key in newivar.keys()}
        difmask = {key: newmask[key] + refmask[key]
                       for key in newmask.keys()}
        difwave = dict(newwave)
        
        print(difwave)
        print(difflux)
        count += 1
        
        if count == 1:
            wefwe
                
    
            

In [ ]:
# print(difivar)
# print(difmask)

In [ ]:
b = difwave['b']
r = difwave['r']
z = difwave['z']

In [ ]:
spec = dict()
spec['b'] =  Lorentz(b,10,5007,4000) + Lorentz(b,10, 4852,3500) + Lorentz(b ,50, 3835,2500)
spec['r'] = Lorentz(r, 10, 6562, 5000) + Lorentz(r, 10, 6732, 5000) 
spec['z'] = Lorentz(z, 10, 9545, 5000) 

# spec['b'] =  scipy.stats.norm(b,10,5007,4000) + scipy.stats.norm(b,10, 4852,3500) + scipy.stats.norm(b ,50, 3835,2500)
# spec['r'] = scipy.stats.norm(r, 10, 6562, 5000) 
# spec['z'] = scipy.stats.norm(z, 10, 9545, 5000)

# y_values = scipy.stats.norm(mean, standard_deviation)

sqivar = dict()
for k in spec.keys():
    sqivar[k] = [0]*len(difivar[k])
    # avg = sum(spec[k])/len(spec[k])
    for i in range(len(spec[k])):
        sqivar[k][i] = math.sqrt(1/difivar[k][i])
difmask = dict()
for k in spec.keys():
    difmask[k] = [0]*len(spec[k])

In [ ]:
plt.plot(difwave['b'], spec['b'])
# plt.show()
plt.plot(difwave['r'], spec['r'])
# plt.show()
plt.plot(difwave['z'], spec['z'])
# plt.show()
plt.plot(difwave['b'],sqivar['b'])
# plt.show()
plt.plot(difwave['r'],sqivar['r'])
# plt.show()
plt.plot(difwave['z'],sqivar['z'])

In [ ]:
perres_filter = perconv_SN(difwave, spec,difivar, difmask)
print(perres_filter)

In [ ]:
linelogic = perres_filter >=2

In [ ]:
print(linelogic)

In [19]:
linetable = line_finder(difwave, spec,difivar,mask,0)

/global/u2/d/divij18/timedomain/desidiff/src/ContinuumFitFilter_desidiff.py:87: RuntimeWarning: divide by zero encountered in true_divide
  HBsigma = 1/np.sqrt(HBivar)
/global/u2/d/divij18/timedomain/desidiff/src/ContinuumFitFilter_desidiff.py:96: RuntimeWarning: divide by zero encountered in true_divide
  Hasigma = 1/np.sqrt(Haivar)


In [20]:
print(linetable)

       Line  Wavelength      Height  e_Height       Sigma     e_Sigma  \
0    Halpha        6562  280.435790  0.297110    5.589958    0.006284   
1     Hbeta        4861   91.311227  0.322891   14.007819    0.060100   
2    Hgamma        4340    0.106124  0.348321   60.582459  577.595254   
3  HeII4686        4686    1.294114  0.128407  118.255886   17.191727   
4  OIII5007        5007  254.770133  0.615179   10.018233    0.035604   
5      NIII        4100    0.315096  0.254082   96.313878    0.348321   
6       SII        6732  268.925940  0.292695    5.645717    0.006078   
7  OIII4959        4959    0.616541  0.646802    8.629740   13.359659   

       Velocity  Chi Square  
0    599.838581  171.274987  
1   2029.116611  111.626751  
2   9829.224849    0.000492  
3  17769.801650  111.626751  
4    598.252760  111.626751  
5  16541.203872    0.000492  
6    590.523401  171.274987  
7    520.325145  111.626751  


In [21]:
Hline_score = Hline_filter(linetable)

In [22]:
print(Hline_score)

0
